<a href="https://colab.research.google.com/github/minson18/ML_final_project/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OverBoostClassifier(n_estimators=100, learning_rate= 0.075, random_state=25) => 0.58977


RUSBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=100, learning_rate= 0.051, random_state=89) 
=> 0.59044

XGBClassifier(n_estimators=110, learning_rate=0.071, random_state=0, max_depth=1, gamma=0, subsample=0.6, colsample_bytree=1) => 0.59014

In [ ]:
pip install imbalanced-ensemble

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minson0218","key":"628656617db3bfef2e291dea28ffec75"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/ML/final project')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC, SVR
from sklearn.metrics import accuracy_score
import math
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from imbalanced_ensemble.ensemble import OverBoostClassifier
from imbalanced_ensemble.ensemble import SelfPacedEnsembleClassifier
import kaggle
from xgboost import XGBClassifier
import os
import random

In [ ]:
train_df = pd.read_csv('train.csv')
train_df['attribute_0']= train_df['attribute_0'].replace(['material_7', 'material_5'],[7, 5])
train_df['attribute_1']= train_df['attribute_1'].replace(['material_8', 'material_5', 'material_6'],[8, 5, 6])
train_df.insert(1, 'measurement_5_wasmissing', 0)
train_df.loc[pd.isna(train_df['measurement_5']),'measurement_5_wasmissing'] = 1
train_df.insert(1, 'measurement_4_wasmissing', 0)
train_df.loc[pd.isna(train_df['measurement_4']),'measurement_4_wasmissing'] = 1
train_df.drop(['id', 'product_code'], axis=1, inplace=True)
train_df = train_df.fillna(0)
train_df_select = train_df[["loading", "measurement_17", "measurement_6", "measurement_4", "measurement_2", "measurement_5_wasmissing", "attribute_3", "attribute_1", "measurement_9", "measurement_4_wasmissing", "measurement_7", "attribute_0", "failure"]]
#train_df = train_df[["loading", "failure"]]
trainData = train_df.to_numpy()
trainData_select = train_df_select.to_numpy()
x_train = trainData[:, :-1]
y_train = trainData[:, -1]
x_train_select = trainData_select[:, :-1]
y_train_select = trainData_select[:, -1]

In [ ]:
test_df = pd.read_csv('test.csv')
test_df['attribute_0']= test_df['attribute_0'].replace(['material_7', 'material_5'],[7, 5])
test_df['attribute_1']= test_df['attribute_1'].replace(['material_7', 'material_5', 'material_6'],[7, 5, 6])
test_df.insert(1, 'measurement_5_wasmissing', 0)
test_df.loc[pd.isna(test_df['measurement_5']),'measurement_5_wasmissing'] = 1
test_df.insert(1, 'measurement_4_wasmissing', 0)
test_df.loc[pd.isna(test_df['measurement_4']),'measurement_4_wasmissing'] = 1
test_df = test_df.drop(['product_code'], axis=1)
test_df = test_df.fillna(0)
test_df_select = test_df[["loading", "measurement_17", "measurement_6", "measurement_4", "measurement_2", "measurement_5_wasmissing", "attribute_3", "attribute_1", "measurement_9", "measurement_4_wasmissing", "measurement_7", "attribute_0"]]
#test_df = test_df[["id", "loading", "attribute_3"]]
#test_df_no_id = test_df_no_id["loading"]
x_test_all = test_df.to_numpy()
x_test_select = test_df_select.to_numpy()
x_test = x_test_all[:, 1:]
test_id = x_test_all[:, 0].astype(int)
print(test_id[:5])

[26570 26571 26572 26573 26574]


In [140]:
m = RUSBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=105, learning_rate= 0.049, random_state=89)
m.fit(x_train, y_train)

RUSBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.049, n_estimators=105, random_state=89)

In [144]:
import pickle
pickle.dump(m, open("try.pickle", 'wb'))

In [145]:
with open('try.pickle', 'rb') as f:
  saved_model = pickle.load(f)

In [150]:
from imbalanced_ensemble.ensemble import OverBoostClassifier
from imbalanced_ensemble.ensemble import RUSBoostClassifier
from xgboost import XGBClassifier
def average(x_train, y_train, x_train_select, y_train_select, x_test, x_test_select):
  model1 = RUSBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=110, learning_rate= 0.052, random_state=10)
  model2 = RUSBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=105, learning_rate= 0.051, random_state=34)
  #model3 = RUSBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=110, learning_rate= 0.049, random_state=10)
  models_featureSelect = [model1, model2]

  modelA = RUSBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=105, learning_rate= 0.049, random_state=89)
  modelB = RUSBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=100, learning_rate= 0.051, random_state=11)
  #modelC = RUSBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=110, learning_rate= 0.049, random_state=89)
  models_missing = [modelA, modelB]

  predictions = []
  for model in models_missing:
    model.fit(x_train, y_train)
    predict = model.predict_proba(x_test)
    predict = predict[:, 1]
    predictions.append(predict)
  
  for model in models_featureSelect:
    model.fit(x_train_select, y_train_select)
    predict = model.predict_proba(x_test_select)
    predict = predict[:, 1]
    predictions.append(predict)

  y_pred = sum(predictions) / len(predictions)
  #print(y_pred.shape)
  
  submit = np.array([test_id, y_pred]).T
  submit = submit.tolist()
  for sub in submit:
    sub[0] = int(sub[0])
    sub[1] = float(sub[1])

  csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
  csv_writer.writerow(["id", "failure"])
  csv_writer.writerows(submit)
  csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
  csv_writer.writerow(["id", "failure"])
  csv_writer.writerows(submit)
  
  #message = f"average of best 2"
  message = f"best4"
  cmd = f"kaggle competitions submit -c tabular-playground-series-aug-2022 -f submission.csv -m '{message}'"
  os.system(cmd)

average(x_train, y_train, x_train_select, y_train_select, x_test, x_test_select)

In [134]:
from imbalanced_ensemble.ensemble import CompatibleBaggingClassifier
from imbalanced_ensemble.ensemble import RUSBoostClassifier
from imbalanced_ensemble.sampler.under_sampling import RandomUnderSampler 
from sklearn.ensemble import AdaBoostClassifier
# In case the high accuracy(0.98) I get in adaboost, n_estimators = 100
# I choose this adaboost model as my_model
def train_XGB(x_train, y_train, n_estimators=110, learning_rate=0.071, random_state=0, max_depth=1, gamma=0, subsample=0.6, colsample_bytree=1, sampler_random_state=0):
    best_model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=random_state, max_depth=max_depth, gamma=gamma, subsample=subsample, colsample_bytree=colsample_bytree)
    #best_model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=random_state, max_depth=max_depth, gamma=gamma, subsample=subsample, colsample_bytree=colsample_bytree)
    sampler = ADASYN(random_state=sampler_random_state)
    X_res, y_res = sampler.fit_resample(x_train, y_train)
    best_model.fit(X_res, y_res)
    return best_model

def train_boost(x_train, y_train, n_estimators=100, learning_rate=0.051, random_state=11, max_depth=1):
    #best_model = BalanceCascadeClassifier(base_estimator=DecisionTreeClassifier(max_depth=max_depth), n_estimators=n_estimators, random_state=random_state)
    best_model = RUSBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=max_depth), n_estimators=n_estimators, learning_rate= learning_rate, random_state=random_state)
    best_model.fit(x_train, y_train)
    return best_model
    
def train_bagging(x_train, y_train, n_estimators=100, learning_rate=1.0, max_samples=1, max_features=1, random_state=0):
    #print(np.unique(y_train).size)
    best_model = CompatibleBaggingClassifier(n_estimators=n_estimators, max_samples=max_samples, max_features=max_features, random_state=random_state)
    best_model.fit(x_train, y_train)
    return best_model

In [136]:
def search(x_train, y_train, x_test, n_estimators=100, learning_rate=0.051, random_state=0, max_depth=1, max_samples=1, max_features=1, gamma=0, subsample=1, colsample_bytree=1, sampler_random_state=0):
  #my_model = train_bagging(x_train, y_train, n_estimators=n_estimators, max_samples=max_samples, max_features=max_features, random_state=random_state)
  #my_model = train_XGB(x_train, y_train, n_estimators=n_estimators, learning_rate=learning_rate, random_state=random_state, max_depth=max_depth, gamma=gamma, subsample=subsample, colsample_bytree=colsample_bytree, sampler_random_state=sampler_random_state)
  my_model = train_boost(x_train, y_train, n_estimators=n_estimators, learning_rate=learning_rate, random_state=random_state, max_depth=1)
  y_pred = my_model.predict_proba(x_test)
  y_pred = y_pred[:, 1]
  submit = np.array([test_id, y_pred]).T

  submit = submit.tolist()
  for sub in submit:
    sub[0] = int(sub[0])
    sub[1] = float(sub[1])

  csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
  csv_writer.writerow(["id", "failure"])
  csv_writer.writerows(submit)
  csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
  csv_writer.writerow(["id", "failure"])
  csv_writer.writerows(submit)
  
  #message = f"CompatibleBaggingClassifier(n_estimators={n_estimators}, max_samples={max_samples}, max_features={max_features}, random_state={random_state})"
  #message = f"XGBClassifier(n_estimators={n_estimators}, learning_rate={learning_rate}, random_state={random_state}, max_depth={max_depth}, gamma={gamma}, subsample={subsample}, colsample_bytree={colsample_bytree})"
  message = f"RUSBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth={max_depth}), n_estimators={n_estimators}, learning_rate={learning_rate}, random_state={random_state}), with feature select & missing 4,5"
  cmd = f"kaggle competitions submit -c tabular-playground-series-aug-2022 -f submission.csv -m '{message}'"
  os.system(cmd)
  #return cmd

In [ ]:
search(x_train, y_train, x_test)

In [111]:
print(np.unique(y_train).size)

2


In [132]:
for n_estimators in [50, 70, 90, 95, 100, 105, 110, 120]:
  #for lr in [1, 0.5, 0.1, 0.05, 0.01, 0.001 ,0.0001]:
    for random_state in range(0, 1):
      search(x_train, y_train, x_test, n_estimators=n_estimators, random_state=random_state)

2
2
2
2
2
2
2
2


In [137]:
for n_estimators in [110]:
  for lr in [0.052]:
    for random_state in range(100):
      search(x_train_select, y_train_select, x_test_select, n_estimators=n_estimators, random_state=random_state, learning_rate=lr)

In [ ]:
#my_model = train_your_model(train_df)
my_model = train_your_model(x_train, y_train)

In [ ]:
y_pred = my_model.predict_proba(x_test)
# np.save("y_pred", y_pred)

In [ ]:
y_pred = y_pred[:, 1]
print(y_pred.shape)

(20775,)


In [ ]:
submit = np.array([test_id, y_pred]).T
print(submit.shape)
submit = submit.tolist()
for sub in submit:
  sub[0] = int(sub[0])
  sub[1] = float(sub[1])

(20775, 2)


In [ ]:
csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
csv_writer.writerow(["id", "failure"])
csv_writer.writerows(submit)
csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
csv_writer.writerow(["id", "failure"])
csv_writer.writerows(submit)

In [ ]:
test_df = pd.read_csv('x_test.csv')
x_test = test_df.to_numpy()

In [ ]:
pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.linear_model import HuberRegressor, LinearRegression
from sklearn.impute import KNNImputer

In [ ]:
def preprocessing(df_train, df_test):
    data = pd.concat([df_train, df_test])
    
    data['m3_missing'] = data['measurement_3'].isnull().astype(np.int8)
    data['m5_missing'] = data['measurement_5'].isnull().astype(np.int8)
    data['area'] = data['attribute_2'] * data['attribute_3']
    feature = [f for f in df_test.columns if f.startswith('measurement') or f =='loading']

    full_fill_dict ={}
    full_fill_dict['measurement_17'] = {
        'A': ['measurement_5','measurement_6','measurement_8'],
        'B': ['measurement_4','measurement_5','measurement_7'],
        'C': ['measurement_5','measurement_7','measurement_8','measurement_9'],
        'D': ['measurement_5','measurement_6','measurement_7','measurement_8'],
        'E': ['measurement_4','measurement_5','measurement_6','measurement_8'],
        'F': ['measurement_4','measurement_5','measurement_6','measurement_7'],
        'G': ['measurement_4','measurement_6','measurement_8','measurement_9'],
        'H': ['measurement_4','measurement_5','measurement_7','measurement_8','measurement_9'],
        'I': ['measurement_3','measurement_7','measurement_8']
    }

    # collect the name of the next 10 best measurement columns sorted by correlation (except 17 already done above):
    col = [col for col in df_test.columns if 'measurement' not in col]+ ['loading','m3_missing','m5_missing']
    a = []
    b =[]
    for x in range(3,17):
        corr = np.absolute(data.drop(col, axis=1).corr()[f'measurement_{x}']).sort_values(ascending=False)
        a.append(np.round(np.sum(corr[1:4]),3)) # we add the 3 first lines of the correlation values to get the "most correlated"
        b.append(f'measurement_{x}')
    c = pd.DataFrame()
    c['Selected columns'] = b
    c['correlation total'] = a
    c = c.sort_values(by = 'correlation total',ascending=False).reset_index(drop = True)
    #print(f'Columns selected by correlation sum of the 3 first rows : ')
    #display(c.head(10))

    for i in range(10):
        measurement_col = 'measurement_' + c.iloc[i,0][12:] # we select the next best correlated column 
        fill_dict = {}
        for x in data.product_code.unique() : 
            corr = np.absolute(data[data.product_code == x].drop(col, axis=1).corr()[measurement_col]).sort_values(ascending=False)
            measurement_col_dic = {}
            measurement_col_dic[measurement_col] = corr[1:5].index.tolist()
            fill_dict[x] = measurement_col_dic[measurement_col]
        full_fill_dict[measurement_col] =fill_dict

    feature = [f for f in data.columns if f.startswith('measurement') or f=='loading']
    nullValue_cols = [col for col in df_train.columns if df_train[col].isnull().sum()!=0]

    for code in data.product_code.unique():
        for measurement_col in list(full_fill_dict.keys()):
            tmp = data[data.product_code == code]
            column = full_fill_dict[measurement_col][code]
            tmp_train = tmp[column+[measurement_col]].dropna(how='any')
            tmp_test = tmp[(tmp[column].isnull().sum(axis=1)==0)&(tmp[measurement_col].isnull())]

            model = HuberRegressor(epsilon=1.9)
            model.fit(tmp_train[column], tmp_train[measurement_col])
            data.loc[(data.product_code==code)&(data[column].isnull().sum(axis=1)==0)&(data[measurement_col].isnull()),measurement_col] = model.predict(tmp_test[column])

        # others NA columns:
        NA = data.loc[data["product_code"] == code,nullValue_cols ].isnull().sum().sum()
        model1 = KNNImputer(n_neighbors=3)
        data.loc[data.product_code==code, feature] = model1.fit_transform(data.loc[data.product_code==code, feature])

    data['measurement_avg'] = data[[f'measurement_{i}' for i in range(3, 17)]].mean(axis=1)
    df_train = data.iloc[:df_train.shape[0],:]
    df_test = data.iloc[df_train.shape[0]:,:]
    features = ['loading', 'attribute_0', 'measurement_17', 'measurement_0', 'measurement_1', 'measurement_2', 'area', 'm3_missing', 'm5_missing', 'measurement_avg']
    
    return df_train, df_test, features

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train, test, features = preprocessing(train_df, test_df)
print(features)
target = train['failure'].copy()

In [ ]:
x_train = train[features].to_numpy()
y_train = target.to_numpy()

In [ ]:
#'reg_lambda': 0.002244159935105443, 'reg_alpha': 6.02990393702363e-08, 'feature_selector': 'cyclic'

In [ ]:
#y_pred = my_model.predict(x_val)
#print(f'Accuracy for my_model {accuracy_score(y_val, y_pred)}')

Accuracy for my_model 0.98
